In [1]:
from typing import List, Optional, NamedTuple
class Rectangle(NamedTuple):
    """Хранит координаты прямоугольника (xmin, ymin) - (xmax, ymax)"""

    xmin: int
    ymin: int
    xmax: int
    ymax: int

    @property
    def w(self) -> int:
        """Ширина"""
        return self.xmax - self.xmin

    @property
    def h(self) -> int:
        """Высота"""
        return self.ymax - self.ymin

    @property
    def square(self) -> float:
        """Площадь"""
        return self.w * self.h


def image_grid(image_w: int, image_h: int,
               window_w: int, window_h: int,
               overlap_w: int, overlap_h: int) -> List[Rectangle]:
    """Рассчитывает координаты прямоугольников для разбиения изображения на блоки
    :param image_w: ширина изображения
    :param image_h: высота изображения
    :param window_w: ширина прямоугольника
    :param window_h: высота прямоугольника
    :param overlap_w: перекрытие прямоугольников по горизонтали
    :param overlap_h: перекрытие прямоугольников по вертикали
    """
    rectangles = []
    # комбинируем вертикальный и горизонтальные разрезы, чтобы получить прямоугольники
    for xmin in cut_points(image=image_w, window=window_w, overlap=overlap_w):
        for ymin in cut_points(image=image_h, window=window_h, overlap=overlap_h):
            rect = Rectangle(xmin=xmin, ymin=ymin, xmax=xmin + window_w, ymax=ymin + window_h)
            rectangles.append(rect)
    # избавляемся от повторов
    return list(set(rectangles))

def cut_points(image: int, window: int, overlap: int) -> List[int]:
    """Точки разрезов изображения (направляющие)"""
    points = []
    offset = window - overlap
    for v in range(0, image - window, offset):
        points.append(v)
    # справа и снизу остается неполный прямоугольник
    # добавим его, отсутпив справа ширину окна и сделаем еще один разрез
    points.append(image - window)
    return points

def image_preprocessing(x):
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x/255. # rescale to [0,1]
    return(x)

def compute_resize_scale(image_shape, min_side=2100, max_side=2100):
    """ Compute an image scale such that the image size is constrained to min_side and max_side.
    Args
        min_side: The image's min side will be equal to min_side after resizing.
        max_side: If after resizing the image's max side is above max_side, resize until the max side is equal to max_side.
    Returns
        A resizing scale.
    """
    (rows, cols, _) = image_shape

    smallest_side = min(rows, cols)

    # rescale the image so the smallest side is min_side
    scale = min_side / smallest_side

    # check if the largest side is now greater than max_side, which can happen
    # when images have a large aspect ratio
    largest_side = max(rows, cols)
    if largest_side * scale > max_side:
        scale = max_side / largest_side

    return scale


def resize_image(img, min_side=2100, max_side=2100):
    """ Resize an image such that the size is constrained to min_side and max_side.
    Args
        min_side: The image's min side will be equal to min_side after resizing.
        max_side: If after resizing the image's max side is above max_side, resize until the max side is equal to max_side.
    Returns
        A resized image.
    """
    # compute scale to resize the image
    scale = compute_resize_scale(img.shape, min_side=min_side, max_side=max_side)

    # resize the image with the computed scale
    img = cv2.resize(img, None, fx=scale, fy=scale)

    return img, scale

def crop_image(img, window_w: int = 800, window_h: int = 800, overlap_w: int = 150, overlap_h: int = 150):
    height_ori, width_ori = img.shape[:2]
    rects = image_grid(image_w=width_ori, image_h=height_ori,
                        window_w=window_w, window_h=window_w,
                        overlap_w=overlap_w, overlap_h=overlap_h)
    
    result = []
    for r in rects:
        crop_img = img[r.ymin:r.ymax, r.xmin:r.xmax]
        dic = {
            "rectangle": r,
            "image": crop_img 
        }
        result.append(dic)
    return result

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import sys
import tensorflow as tf
from tensorflow.keras import backend as K
import segmentation_models as sm
from datetime import datetime
import time
import cv2
print(tf.__version__)
K.set_image_data_format('channels_last')

Segmentation Models: using `tf.keras` framework.
2.3.0


In [3]:
model = sm.Linknet(backbone_name='resnet18', classes=1,encoder_weights='imagenet', encoder_freeze=True)
model.compile('Adam', 
              loss=sm.losses.binary_focal_dice_loss,
              metrics=[sm.metrics.iou_score, sm.metrics.f1_score])
model.load_weights('linknet_resnet_18_800_full2.h5')

In [5]:
def run_detection_image(in_filepath, out_filepath, out_maskpath):
    image = cv2.imread(in_filepath)
    draw = image.copy()
    raw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    image, scale = resize_image(image)
    (rows, cols, _) = image.shape
    r_mask = np.zeros((rows, cols, 1), np.float32)
    
    # crop image
    crops = crop_image(image)
    masks = []
    
    # detect loop
    start = time.time()
    for i in range(0, len(crops)):
        img = image_preprocessing(crops[i]['image'])
        mask = model.predict_on_batch(np.expand_dims(img, axis=0))
        masks.append(mask[0])
    print("processing time: ", time.time() - start) 
    
    # post process loop
    for i in range(0, len(crops)):
        mask = masks[i]
        rect = crops[i]['rectangle']
        r_mask_roi = r_mask[rect.ymin:rect.ymax,rect.xmin:rect.xmax,:mask.shape[2]]
        r_mask[rect.ymin:rect.ymax,rect.xmin:rect.xmax,:mask.shape[2]] = np.maximum(r_mask_roi, mask)
        
    
    r_mask = ((r_mask > 0.9) * 255).astype("uint8")
    contours, _  = cv2.findContours(r_mask, 1, 2)
    contours_poly = [None]*len(contours)
    boundRect = [None]*len(contours)
    for i, c in enumerate(contours):
        contours_poly[i] = cv2.approxPolyDP(c, 3, True)
        boundRect[i] = cv2.boundingRect(contours_poly[i])

    for i in range(len(contours)):
        color = (0, 0, 255)
        cv2.rectangle(draw, \
            (int(boundRect[i][0] / scale), int(boundRect[i][1] / scale)), \
            (int((boundRect[i][0]+boundRect[i][2]) / scale), int((boundRect[i][1]+boundRect[i][3]) / scale)), \
            color, 5)
        
    if len(contours) > 0:
        cv2.imwrite(out_filepath, draw)
        cv2.imwrite(out_maskpath, r_mask)

In [6]:
test_in_base_dir = '/home/gosha20777/files/datasets/tests/test2/input'
test_out_base_dir = '/home/gosha20777/files/datasets/tests/test2/output'
save_base_dir = '/home/gosha20777/files/datasets/tests/test2/output/linknet_resnet18_2100x2100_full'
if not os.path.isdir(save_base_dir):
    os.mkdir(save_base_dir)
save_mask_dir = '/home/gosha20777/files/datasets/tests/test2/output/linknet_resnet18_2100x2100_full/mask'
if not os.path.isdir(save_mask_dir):
    os.mkdir(save_mask_dir)

tests = os.listdir(test_in_base_dir)
for test_name in tests:
    print(test_name)
    in_img_dir = os.path.join(test_in_base_dir, test_name)
    out_img_dir = os.path.join(save_base_dir, test_name)
    if not os.path.isdir(out_img_dir):
        os.mkdir(out_img_dir)
        
    out_mask_dir = os.path.join(save_mask_dir, test_name)
    if not os.path.isdir(out_mask_dir):
        os.mkdir(out_mask_dir)
    
    for img in os.listdir(in_img_dir):
        run_detection_image(os.path.join(in_img_dir, img), 
                            os.path.join(out_img_dir, img),
                            os.path.join(out_mask_dir, img))

false_posetive_kld
processing time:  2.0098071098327637
processing time:  1.639794111251831
processing time:  1.5589303970336914
processing time:  1.5746207237243652
processing time:  1.5898759365081787
processing time:  1.5929505825042725
processing time:  1.5976405143737793
processing time:  1.5952951908111572
processing time:  1.5982396602630615
processing time:  1.5801301002502441
processing time:  1.610456943511963
processing time:  1.6030879020690918
processing time:  1.6034936904907227
processing time:  1.5942339897155762
processing time:  1.6032228469848633
processing time:  1.6532938480377197
processing time:  1.6597375869750977
processing time:  1.682326316833496
processing time:  1.625948190689087
processing time:  1.6406636238098145
processing time:  1.6407175064086914
processing time:  1.7610423564910889
processing time:  1.6858000755310059
processing time:  1.6866943836212158
processing time:  1.717618465423584
processing time:  1.7174124717712402
processing time:  1.7744

processing time:  1.6788485050201416
processing time:  1.6942174434661865
processing time:  1.7433593273162842
processing time:  1.6858861446380615
processing time:  1.6832566261291504
processing time:  1.6870920658111572
processing time:  1.6895864009857178
processing time:  1.691471815109253
processing time:  1.6871087551116943
processing time:  1.7038443088531494
processing time:  1.7101752758026123
processing time:  1.6857209205627441
processing time:  1.6856005191802979
processing time:  1.6883940696716309
processing time:  1.758845567703247
processing time:  1.6907556056976318
processing time:  1.6890501976013184
processing time:  1.6810967922210693
processing time:  1.6773459911346436
processing time:  1.6862378120422363
processing time:  1.680237054824829
processing time:  1.6902949810028076
processing time:  1.7796330451965332
processing time:  1.8740346431732178
processing time:  1.7930023670196533
processing time:  1.7720954418182373
processing time:  1.821390151977539
proce

processing time:  1.6720788478851318
processing time:  1.6896576881408691
processing time:  1.6926155090332031
processing time:  1.6775362491607666
processing time:  1.6674394607543945
processing time:  1.674593448638916
processing time:  1.6777064800262451
processing time:  1.6800892353057861
processing time:  1.680224895477295
processing time:  1.6735081672668457
processing time:  1.6774654388427734
processing time:  1.6803233623504639
processing time:  1.7040951251983643
processing time:  1.7015273571014404
processing time:  1.689232349395752
processing time:  1.6908767223358154
processing time:  1.6831185817718506


In [8]:
run_detection_image('/home/gosha20777/files/datasets/tests/test1/input/gpa_test/001.jpg', 'out.jpg', 'out_mask.jpg')

processing time:  1.5652763843536377
